## This is my take on DeepSDF

To be completely honest, I'm not sure if that's what I was supposed to do. Maybe simple geometric solution would work, but here we go :)

That's basically my implementation of [DeepSDF](https://github.com/facebookresearch/DeepSDF/).

To give them credits, I looked at their implementation, while doing mine (not that it was very helpful though). So I'd recommend diving into their repo for a more sophisticated approach.

### P.S.
Take this project with a grain of salt, since for real-world applications it would be much more feasible to get a pretrained DeepSDF (for example from [here](https://github.com/marian42/shapegan/tree/pretrained-deepsdf-shapenet/examples/deepsdf-shapenet-pretrained)) and produce much better scores. This notebook is specifically about my experience in discovering the mesh & SDF kind-of data and working with shape embeddings, aka DeepSDF.

## Data
You can download the prepared data [here](https://drive.google.com/drive/folders/1AE_mohNpxRg3JXoBX2oiN-8xaMcGYYuX?usp=sharing)

## TL;DR

### What I got

- Modeled and trained a DeepSDF model 
- 8 * float64 = 64 byte representations
- 4.6 ms for a batch size of 1024
- Around 0.94 validation f1 occupancy with sdf < 1e-3 (very dirty considering the sign distribution though)
- Pretty bad inside-shape modeling

### What could help and improve my results
- **More flexible and accurate solver and better data sampling for inside-points modeling (maybe sampling negative SDFs more agressively)**
- **More RAM and time for modeling a much bigger dataset (30 train models doesn't sound like a good amount of information for learning the latent shape space)**
- **Adequate validation (I believe all the hard stuff is done in this notebook, and it shouldn't be difficult to push the f1 score up from this point, it's just a routine work with data)**
- Doing the loss clamping (as proposed in the original paper) for a better surface and occupancy modeling
- More experiments with an architecture and training (weight decay as proposed in DeepSDF, lr scheduling)

In [1]:
import os
import torch
import trimesh
import numpy as np

from torch import nn
from tqdm import tqdm
from itertools import chain
from sklearn.metrics import f1_score
from torch.utils.data import Dataset, DataLoader
from mesh_to_sdf import sample_sdf_near_surface

In [2]:
# Subsampling the train and test data, since my PC isn't from NASA
SUBSAMPLE_TRAIN_VALID = 30
SUBSAMPLE_TEST = 5

# Network params
NUM_EPOCHS_TRAIN = 50
NUM_EPOCHS_TEST = 30

BATCH_SIZE = 1024
EMBEDDING_DIM = 8  # Higher values neither help nor make sence considering the amount of subsampled shapes
DROPOUT = 0.2
HIDDEN_DIMS = [64] * 5

In [3]:
class SimpleBlock(nn.Module):

    ''' Linear -> BN -> Dropout -> LeakyReLU '''
    
    def __init__(self, inp_dim, out_dim, dropout=0.2):
        super(SimpleBlock, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(inp_dim, out_dim),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(dropout),
            nn.LeakyReLU()
        )
        
    def forward(self, X):
        return self.model(X)
    
class SDFDecoder(nn.Module):
    
    ''' MLP decoder of SimpleBlock layers with tahn activation in prediction head '''
    
    def __init__(self, inp_dim, hidden_dims):
        super(SDFDecoder, self).__init__()
        
        self.model = nn.Sequential(
            SimpleBlock(inp_dim, hidden_dims[0]),
            *[SimpleBlock(i, o) for i, o in zip(hidden_dims, hidden_dims[1:])],
            nn.Linear(hidden_dims[-1], 1),
            nn.Tanh()
        )
        
    def forward(self, X):
        return self.model(X)
    
    
class SDFDataset(Dataset):
    
    ''' Dataset class for producing [latent_vector + xyz] -> SDF mapping'''
    
    def __init__(self, filenames, n_samples, encoder):
        """ Supports loading and sampling from @filenames paths but I didn't use it in the end """
        self.encoder = encoder
        self.indices = []
        self.X = torch.Tensor([])
        self.y = torch.Tensor([])
        
        for i, filepath in enumerate(tqdm(filenames)):
            self.indices.extend(n_samples * [i])
            mesh = trimesh.load(filepath)
            points, sdf = sample_sdf_near_surface(mesh, number_of_points=n_samples, sign_method='depth')
            self.X = torch.cat((self.X, torch.from_numpy(points)))
            self.y = torch.cat((self.y, torch.from_numpy(sdf)))
        
        self.indices = torch.Tensor(self.indices).int()
        
    # A bit tinky-winky save/load, zipping would make more sense but whatever
    def save(self, indices_name, X_name, y_name):
        torch.save(self.indices, indices_name)
        torch.save(self.X, X_name)
        torch.save(self.y, y_name)
    
    def load(self, indices_name, X_name, y_name):
        self.indices = torch.load(indices_name)
        self.X = torch.load(X_name)
        self.y = torch.load(y_name)
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        """ Get encoded latent vector + xyz pair with corresponding SDF """
        return torch.cat((self.encoder(self.indices[idx]), self.X[idx])), self.y[idx]

In [4]:
# Encoders initialization
train_encoder = nn.Embedding(SUBSAMPLE_TRAIN_VALID, EMBEDDING_DIM)
test_encoder = nn.Embedding(SUBSAMPLE_TEST, EMBEDDING_DIM)

# We'll load the prepaired data in the next cell
train_dataset = SDFDataset([], None, train_encoder)
valid_dataset = SDFDataset([], None, train_encoder)
test_dataset = SDFDataset([], None, test_encoder)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [5]:
# Load already processed data
train_dataset.load('processed_data/train_idx.pt', 'processed_data/train_X.pt', 'processed_data/train_y.pt')
valid_dataset.load('processed_data/valid_idx.pt', 'processed_data/valid_X.pt', 'processed_data/valid_y.pt')
test_dataset.load('processed_data/test_idx.pt', 'processed_data/test_X.pt', 'processed_data/test_y.pt')

In [6]:
# Again, a bit tinky-winky subsampling
train_ind = train_dataset.indices < SUBSAMPLE_TRAIN_VALID
valid_ind = valid_dataset.indices < SUBSAMPLE_TRAIN_VALID
test_ind = test_dataset.indices < SUBSAMPLE_TEST

train_dataset.indices = train_dataset.indices[train_ind]
train_dataset.X = train_dataset.X[train_ind]
train_dataset.y = train_dataset.y[train_ind]

valid_dataset.indices = valid_dataset.indices[valid_ind]
valid_dataset.X = valid_dataset.X[valid_ind]
valid_dataset.y = valid_dataset.y[valid_ind]

test_dataset.indices = test_dataset.indices[test_ind]
test_dataset.X = test_dataset.X[test_ind]
test_dataset.y = test_dataset.y[test_ind]

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
print(f'{len(train_dataset)=}', f'{len(valid_dataset)=}', f'{len(test_dataset)=}', sep='\n')

len(train_dataset)=3000000
len(valid_dataset)=600000
len(test_dataset)=400000


In [9]:
decoder = SDFDecoder(EMBEDDING_DIM + 3, HIDDEN_DIMS)

criterion = nn.L1Loss()
optimizer = torch.optim.Adam(chain(train_encoder.parameters(), decoder.parameters()), lr = 0.0005)

In [10]:
train_encoder.train()
decoder.train()

for epoch in range(NUM_EPOCHS_TRAIN):
    pbar = tqdm(train_dataloader)
    for X, y in pbar:
        
        optimizer.zero_grad()
        
        out = decoder(X).squeeze()
    
        loss = criterion(out, y)
        pbar.set_description(f'Batch loss: {loss.item()}, f1: {f1_score(y>0, out>0)}', refresh=True)

        # Backward pass
        loss.backward()
        optimizer.step()
        
    valid_y, valid_out = map(torch.cat, zip(*[(y, decoder(X).squeeze()) for X, y in valid_dataloader]))
    
    ind = valid_y.abs() < 1e-3
    occupancy_y = valid_y[ind] > 0
    occupancy_out = valid_out[ind] > 0

    print(f'Valid {epoch=}, loss={criterion(valid_out, valid_y).item():.4f}, f1={f1_score(valid_y>0, valid_out>0):.3f},\
     occupancy_f1={f1_score(occupancy_y, occupancy_out):.3f}')

Batch loss: 0.04344232380390167, f1: 0.8998435054773084: 100%|█| 2930/2930 [01:39<00:00, 29.47it/


Valid epoch=0, loss=0.0395, f1=0.898,     occupancy_f1=0.946


Batch loss: 0.04686539247632027, f1: 0.9051321928460342: 100%|█| 2930/2930 [01:26<00:00, 33.68it/


Valid epoch=1, loss=0.0396, f1=0.898,     occupancy_f1=0.946


Batch loss: 0.013404348865151405, f1: 0.9000000000000001: 100%|█| 2930/2930 [01:29<00:00, 32.62it


...

Valid epoch=47, loss=0.0066, f1=0.887,     occupancy_f1=0.932


Batch loss: 0.005524642299860716, f1: 0.8930817610062893: 100%|█| 2930/2930 [01:34<00:00, 30.86it


Valid epoch=48, loss=0.0066, f1=0.894,     occupancy_f1=0.941


Batch loss: 0.006987675558775663, f1: 0.8937844217151848: 100%|█| 2930/2930 [01:36<00:00, 30.36it


Valid epoch=49, loss=0.0065, f1=0.891,     occupancy_f1=0.937


In [11]:
# Again, I don't have a NASA PC
del train_dataset, valid_dataset, train_dataloader, valid_dataloader

In [12]:
# Freeze the decoder layers to finetune the test representations
decoder.requires_grad = False
for param in decoder.parameters():
    param.requires_grad = False

In [13]:
test_criterion = nn.L1Loss()
test_optimizer = torch.optim.Adam(test_encoder.parameters(), lr = 0.0003)

In [14]:
test_encoder.train()

for epoch in range(NUM_EPOCHS_TEST):
    pbar = tqdm(test_dataloader)
    for X, y in pbar:
        
        test_optimizer.zero_grad()
        
        out = decoder(X).squeeze()
        
        test_loss = test_criterion(out, y)
        pbar.set_description(f'Batch loss: {test_loss.item()}, f1: {f1_score(y>0, out>0)}', refresh=True)

        # Backward pass
        test_loss.backward()
        test_optimizer.step()
    
    test_y, test_out = map(torch.cat, zip(*[(y, decoder(X).squeeze()) for X, y in test_dataloader]))
    ind = test_y < 1e-3
    occupancy_y = test_y[ind] > 0
    occupancy_out = test_out[ind] > 0

    print(f'Test {epoch=}, loss={criterion(test_out, test_y).item():.4f}, f1={f1_score(test_y>0, test_out>0):.3f},\
    occupancy_f1={f1_score(occupancy_y, occupancy_out):.3f}')

Batch loss: 0.06008940190076828, f1: 0.6603550295857987: 100%|█| 391/391 [00:12<00:00, 32.56it/s]


Test epoch=0, loss=0.1445, f1=0.457,    occupancy_f1=0.319


Batch loss: 0.11212427914142609, f1: 0.4747191011235955: 100%|█| 391/391 [00:11<00:00, 33.88it/s]


Test epoch=1, loss=0.0856, f1=0.568,    occupancy_f1=0.437


Batch loss: 0.026577701792120934, f1: 0.8295454545454546: 100%|█| 391/391 [00:11<00:00, 32.85it/s


...

Test epoch=27, loss=0.0110, f1=0.888,    occupancy_f1=0.710


Batch loss: 0.008971111848950386, f1: 0.8737864077669903: 100%|█| 391/391 [00:11<00:00, 32.81it/s


Test epoch=28, loss=0.0108, f1=0.888,    occupancy_f1=0.710


Batch loss: 0.009782833978533745, f1: 0.8990509059534081: 100%|█| 391/391 [00:11<00:00, 33.92it/s


Test epoch=29, loss=0.0107, f1=0.888,    occupancy_f1=0.710


In [15]:
len(occupancy_y), len(occupancy_out)

(171612, 171612)

In [16]:
occupancy_y.sum(), occupancy_out.sum()

(tensor(95473), tensor(167854))

In [17]:
batch = next(iter(test_dataloader))[0]
len(batch)

1024

In [18]:
%timeit decoder(batch)

4.63 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
